In [1]:
# How to download and clean Fama-French data
# https://www.codingfinance.com/post/2019-07-01-analyze-ff-factor-python/
# https://randlow.github.io/posts/finance-economics/pandas-datareader-KF/

import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
import datetime
import pickleshare
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
# get the list of available datasets
datasets = get_available_datasets()
print('No. of datasets:{}'.format(len(datasets)))
#datasets

No. of datasets:297


In [3]:
datasets_factors_5F = [dataset for dataset in datasets if '_5_' in dataset and 'Factor' in dataset]
datasets_factors_3F = [dataset for dataset in datasets if '_3_' in dataset and 'Factor' in dataset]

datasets_factors_weekly = [dataset for dataset in datasets if 'weekly' in dataset and 'Factor' in dataset]


datasets_factors = [dataset for dataset in datasets if 'Factor' in dataset in dataset]
datasets_factors

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'F-F_Momentum_Factor',
 'F-F_Momentum_Factor_daily',
 'F-F_ST_Reversal_Factor',
 'F-F_ST_Reversal_Factor_daily',
 'F-F_LT_Reversal_Factor',
 'F-F_LT_Reversal_Factor_daily',
 'Developed_3_Factors',
 'Developed_3_Factors_Daily',
 'Developed_ex_US_3_Factors',
 'Developed_ex_US_3_Factors_Daily',
 'Europe_3_Factors',
 'Europe_3_Factors_Daily',
 'Japan_3_Factors',
 'Japan_3_Factors_Daily',
 'Asia_Pacific_ex_Japan_3_Factors',
 'Asia_Pacific_ex_Japan_3_Factors_Daily',
 'North_America_3_Factors',
 'North_America_3_Factors_Daily',
 'Developed_5_Factors',
 'Developed_5_Factors_Daily',
 'Developed_ex_US_5_Factors',
 'Developed_ex_US_5_Factors_Daily',
 'Europe_5_Factors',
 'Europe_5_Factors_Daily',
 'Japan_5_Factors',
 'Japan_5_Factors_Daily',
 'Asia_Pacific_ex_Japan_5_Factors',
 'Asia_Pacific_ex_Japan_5_Factors_Daily',

In [4]:
#create dataframe with elements of datasets_factors but not in datasets_5_factors and datasets_3_factors
datasets_factors_others = [dataset for dataset in datasets_factors if dataset not in datasets_factors_5F and dataset not in datasets_factors_3F]
datasets_factors_others

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Momentum_Factor',
 'F-F_Momentum_Factor_daily',
 'F-F_ST_Reversal_Factor',
 'F-F_ST_Reversal_Factor_daily',
 'F-F_LT_Reversal_Factor',
 'F-F_LT_Reversal_Factor_daily',
 'Developed_Mom_Factor',
 'Developed_Mom_Factor_Daily',
 'Developed_ex_US_Mom_Factor',
 'Developed_ex_US_Mom_Factor_Daily',
 'Europe_Mom_Factor',
 'Europe_Mom_Factor_Daily',
 'Japan_Mom_Factor',
 'Japan_Mom_Factor_Daily',
 'Asia_Pacific_ex_Japan_MOM_Factor',
 'Asia_Pacific_ex_Japan_MOM_Factor_Daily',
 'North_America_Mom_Factor',
 'North_America_Mom_Factor_Daily',
 'Emerging_MOM_Factor']

In [5]:
print(f"Number of datasets_factors: {len(datasets_factors)}")                   # 49
print(f"Number of datasets_factors_5F: {len(datasets_factors_5F)}")             # 15
print(f"Number of datasets_factors_3F: {len(datasets_factors_3F)}")             # 12
print(f"Number of datasets_factors_others: {len(datasets_factors_others)}")     # 22
print(f"Number of datasets_factors_weekly: {len(datasets_factors_weekly)}")     # 1

Number of datasets_factors: 49
Number of datasets_factors_5F: 15
Number of datasets_factors_3F: 12
Number of datasets_factors_others: 22
Number of datasets_factors_weekly: 1


In [6]:
# Fama-French 3F Model & Fama-French 5F Model

# Selecting the dataset of interest in string format
ff3_dataset_name = 'F-F_Research_Data_Factors'
ff5_dataset_name = 'F-F_Research_Data_5_Factors_2x3'
# start = date_format("2010-1-1"", %Y-%m-%d")
start = datetime.datetime(1979, 12, 31)

ff3 = web.DataReader(ff3_dataset_name, 'famafrench', start)[0]
ff5 = web.DataReader(ff5_dataset_name, 'famafrench', start)[0]

#convert in decimal format
ff3 = ff3.apply(lambda x: x/ 100)
ff5 = ff5.apply(lambda x: x/ 100)

#convert PeriodDtype index to datestamp index and end of moonth
ff3.index = ff3.index.to_timestamp() + pd.offsets.MonthEnd()
ff5.index = ff5.index.to_timestamp() + pd.offsets.MonthEnd()
#ff3.index = ff3.index 

#rename columns
ff3_name = 'Fama-French 3-Factor Model'
ff5_name = 'Fama-French 5-Factor Model'

# Display two lines of headers with a list of lists
ff3_superheader = [ff3_name] * len(ff3.columns)
ff3_headers = list(ff3.columns)
ff5_superheader = [ff5_name] * len(ff5.columns)
ff5_headers = list(ff5.columns)
ff3_two_lines_headers = [ff3_superheader, ff3_headers]
ff5_two_lines_headers = [ff5_superheader, ff5_headers]
# ff3.columns = ff3_two_lines_headers
# ff5.columns = ff5_two_lines_headers

#concatenate the two dataframes into one with subcategories in columns and add dataset name in columns header
ff3_5 = pd.concat([ff3, ff5], axis=1)

# concatenate the two headers into one horizontal header with ff3_two_lines_headers and ff5_two_lines_headers
ff3_5.columns = [ff3_superheader + ff5_superheader, ff3_headers + ff5_headers]
#ff3_5.columns = pd.MultiIndex.from_arrays([ff3_superheader + ff5_superheader, ff3_headers + ff5_headers])
ff3_5

C:\Users\phil7\AppData\Local\Temp\ipykernel_39340\2612191043.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = web.DataReader(ff3_dataset_name, 'famafrench', start)[0]
C:\Users\phil7\AppData\Local\Temp\ipykernel_39340\2612191043.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = web.DataReader(ff3_dataset_name, 'famafrench', start)[0]
C:\Users\phil7\AppData\Local\Temp\ipykernel_39340\2612191043.py:10: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5 = web.DataReader(ff5_dataset_name, 'famafrench', start)[0]
C:\Use

Fama-French 3-Factor Model                          \
                               Mkt-RF     SMB     HML      RF   
Date                                                            
1979-12-31                     0.0179  0.0417 -0.0210  0.0095   
1980-01-31                     0.0551  0.0162  0.0175  0.0080   
1980-02-29                    -0.0122 -0.0185  0.0061  0.0089   
1980-03-31                    -0.1290 -0.0664 -0.0101  0.0121   
1980-04-30                     0.0397  0.0105  0.0106  0.0126   
...                               ...     ...     ...     ...   
2023-08-31                    -0.0239 -0.0316 -0.0106  0.0045   
2023-09-30                    -0.0524 -0.0251  0.0152  0.0043   
2023-10-31                    -0.0319 -0.0387  0.0019  0.0047   
2023-11-30                     0.0884 -0.0002  0.0164  0.0044   
2023-12-31                     0.0485  0.0635  0.0494  0.0043   

           Fama-French 5-Factor Model                                          
                               Mkt-RF     SMB     HML     RMW     CMA      RF  
Date                                                                           
1979-12-31                     0.0179  0.0432 -0.0210 -0.0070 -0.0092  0.0095  
1980-01-31                     0.0551  0.0183  0.0175 -0.0170  0.0164  0.0080  
1980-02-29                    -0.0122 -0.0157  0.0061  0.0004  0.0268  0.0089  
1980-03-31                    -0.1290 -0.0693 -0.0101  0.0146 -0.0119  0.0121  
1980-04-30                     0.0397  0.0105  0.0106 -0.0210  0.0029  0.0126  
...                               ...     ...     ...     ...     ...     ...  
2023-08-31                    -0.0239 -0.0365 -0.0106  0.0343 -0.0237  0.0045  
2023-09-30                    -0.0524 -0.0180  0.0152  0.0186 -0.0083  0.0043  
2023-10-31                    -0.0319 -0.0404  0.0019  0.0246 -0.0066  0.0047  
2023-11-30                     0.0884 -0.0012  0.0164 -0.0391 -0.0100  0.0044  
2023-12-31                     0.0485  0.0732  0.0494 -0.0307  0.0132  0.0043  

[529 rows x 10 columns]

In [7]:
ff3

,Mkt-RF,SMB,HML,RF
Date,,,,
1979-12-31,0.0179,0.0417,-0.0210,0.0095
1980-01-31,0.0551,0.0162,0.0175,0.0080
1980-02-29,-0.0122,-0.0185,0.0061,0.0089
1980-03-31,-0.1290,-0.0664,-0.0101,0.0121
1980-04-30,0.0397,0.0105,0.0106,0.0126
...,...,...,...,...
2023-08-31,-0.0239,-0.0316,-0.0106,0.0045
2023-09-30,-0.0524,-0.0251,0.0152,0.0043
2023-10-31,-0.0319,-0.0387,0.0019,0.0047


In [8]:
#last date of the dataset
ff_last = ff3.index[-1]
ff_last

Timestamp('2023-12-31 00:00:00')

In [18]:
# get time series of FCNTX
asset_prices = yf.download("FCNTX")
# stop at ff_last
asset_prices = asset_prices[:ff_last]
asset_prices = asset_prices[['Close']]
asset_prices.columns = ['Asset']

asset_prices.tail()

[*********************100%%**********************]  1 of 1 completed


,Asset
Date,
2023-12-22,16.040001
2023-12-26,16.090000
2023-12-27,16.139999
2023-12-28,16.150000
2023-12-29,16.090000


In [31]:
asset_prices_monthly = asset_prices.resample('M').last()
asset_prices_monthly = asset_prices_monthly[:'2019-06-30']
ff3 = ff3['1980-02-29':'2019-06-30']
asset = asset_prices_monthly.pct_change().dropna()

In [35]:
asset['Asset-RF'] = asset['Asset'] - ff3['RF']
asset

,Asset,Asset-RF
Date,,
1980-02-29,-0.016874,-0.025774
1980-03-31,-0.089431,-0.101531
1980-04-30,0.017857,0.005257
1980-05-31,0.078947,0.070847
1980-06-30,0.011743,0.005643
...,...,...
2019-02-28,0.014938,0.013138
2019-03-31,0.022077,0.020177
2019-04-30,0.048800,0.046700


In [36]:
# Run the multiple regression model
#dependent variable
y = asset['Asset-RF'][-50:]
#independent variables
x = ff3['Mkt-RF'][-50:]
x = sm.add_constant(x)
#model
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Asset-RF   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     208.1
Date:                Sun, 18 Feb 2024   Prob (F-statistic):           4.45e-19
Time:                        21:51:15   Log-Likelihood:                 126.73
No. Observations:                  50   AIC:                            -249.5
Df Residuals:                      48   BIC:                            -245.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0033      0.003     -1.177      0.2

In [38]:
# Run the multiple regression model
#dependent variable
y = asset['Asset-RF'].loc['1980-01-31':'2019-05-31']
#independent three variables
x = ff3[['Mkt-RF', 'SMB', 'HML']].loc['1980-01-31':'2019-05-31']
x = sm.add_constant(x)
#model
model = sm.OLS(y, x)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:               Asset-RF   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     400.6
Date:                Sun, 18 Feb 2024   Prob (F-statistic):          7.80e-129
Time:                        21:56:38   Log-Likelihood:                 1071.3
No. Observations:                 472   AIC:                            -2135.
Df Residuals:                     468   BIC:                            -2118.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0028      0.001     -2.336      0.0

In [39]:
model.fit().params

const    -0.002750
Mkt-RF    0.888673
SMB      -0.002831
HML      -0.101656
dtype: float64

In [42]:
pd.concat([x, y], axis=1)

,const,Mkt-RF,SMB,HML,Asset-RF
Date,,,,,
1980-02-29,1.0,-0.0122,-0.0185,0.0061,-0.025774
1980-03-31,1.0,-0.1290,-0.0664,-0.0101,-0.101531
1980-04-30,1.0,0.0397,0.0105,0.0106,0.005257
1980-05-31,1.0,0.0526,0.0213,0.0038,0.070847
1980-06-30,1.0,0.0306,0.0166,-0.0076,0.005643
...,...,...,...,...,...
2019-01-31,1.0,0.0840,0.0290,-0.0046,0.092360
2019-02-28,1.0,0.0340,0.0205,-0.0267,0.013138
2019-03-31,1.0,0.0110,-0.0305,-0.0417,0.020177
